In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
import tkinter as tk
from tkinter import filedialog
from tkinter import Label
from PIL import Image, ImageTk


yolo_model = YOLO('yolov8l.pt')


age_net = cv2.dnn.readNetFromCaffe('models/deploy_age2.prototxt', 'models/age_net.caffemodel')
gender_net = cv2.dnn.readNetFromCaffe('models/deploy_gender2.prototxt', 'models/gender_net (1).caffemodel')


age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
gender_list = ['Male', 'Female']


In [12]:
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

def classify_age_and_gender(face):
    face_blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
    gender_net.setInput(face_blob)
    gender_preds = gender_net.forward()
    gender = gender_list[gender_preds[0].argmax()]

    age_net.setInput(face_blob)
    age_preds = age_net.forward()
    age = age_list[age_preds[0].argmax()]

    return gender, age

def detect_shirt_color(face):
    hsv = cv2.cvtColor(face, cv2.COLOR_BGR2HSV)
    mask_white = cv2.inRange(hsv, (0, 0, 200), (180, 20, 255))
    mask_black = cv2.inRange(hsv, (0, 0, 0), (180, 255, 30))

    if cv2.countNonZero(mask_white) > cv2.countNonZero(mask_black):
        return 'white'
    elif cv2.countNonZero(mask_black) > cv2.countNonZero(mask_white):
        return 'black'
    else:
        return 'other'

def apply_shirt_rules(gender, age, shirt_color):
    if shirt_color == 'white':
        age = '(23)'
    elif shirt_color == 'black':
        age = 'child'
    return gender, age

def process_image(image_path):
    image = cv2.imread(image_path)
    results = yolo_model(image)[0]

    if len(results.boxes) < 2:
        return "Less than 2 people detected"

    for box in results.boxes:
        if int(box.cls) != 0:  
            continue

        xmin, ymin, xmax, ymax = map(int, box.xyxy[0].tolist())
        face = image[ymin:ymax, xmin:xmax]

        try:
            gender, age = classify_age_and_gender(face)
            shirt_color = detect_shirt_color(face)
            gender, age = apply_shirt_rules(gender, age, shirt_color)
            cv2.putText(image, f'{gender}, {age}, {shirt_color}', (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 1)
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        except Exception as e:
            print(f"Error processing person: {e}")

    return image

def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        processed_image = process_image(file_path)
        if isinstance(processed_image, str):
            result_label.config(text=processed_image)
        else:
            image = Image.fromarray(cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB))
            image = ImageTk.PhotoImage(image)
            panel.config(image=image)
            panel.image = image


root = tk.Tk()
root.title("Gender and Age Detection")


upload_button = tk.Button(root, text="Upload Image", command=upload_image)
upload_button.pack()


result_label = Label(root, text="")
result_label.pack()


panel = Label(root)
panel.pack()


root.mainloop()


0: 512x640 1 person, 6 cups, 3 chairs, 1 dining table, 1033.0ms
Speed: 6.0ms preprocess, 1033.0ms inference, 3.0ms postprocess per image at shape (1, 3, 512, 640)
